In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM, RobertaTokenizerFast
from transformers.trainer import TrainingArguments, Trainer
import logging
from utils import metrics
import datasets
import mlflow
from utils import container, text_utils
import tqdm

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [3]:
pre_trained_model_name = 'roberta-base'
logger.critical("Build pre-trained model {}".format(pre_trained_model_name))
base_pre_trained_model_path = '/home/ubuntu/likun/nlp_pretrained/{}'.format(pre_trained_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_pre_trained_model_path)
# tokenizer = RobertaTokenizerFast.from_pretrained(base_pre_trained_model_path)

Build pre-trained model roberta-base


In [4]:
mlflow.set_tracking_uri("http://10.10.111.130:5005")
mlflow.set_experiment("few-shot-finetune")

mlflow_tags = {
    "paper": "Diverse Few-Shot Text Classification with Multiple Metrics",
    "dl_frame": "huggingface-pytorch",
    "pretrain-model": pre_trained_model_name,
    "mlflow.runName": "few-shot-prompt-arsc"
}

In [10]:
logger.critical("Build Training and validating dataset")
dataset_args = {
    "dataset_name": "ARSC",
    "data_cache_dir": "/home/ubuntu/likun/huggingface_dataset",
    "train_size": 192,
    "per_class_train_size": 32,
    "val_size": 200,
    "test_size": 500,
    "max_length": 100,
    "shuffle": True,
    "val_source": "no"  # no, trian, val, test
}
mlflow_tags.update(dataset_args)
# dataset = datasets.load_dataset(dataset_args['dataset_name'],cache_dir=dataset_args['data_cache_dir'])
dataset = datasets.load_dataset('csv', data_files={'train': ['ARSC_train.csv'], 'test': ['ARSC_test.csv']})
# dataset = datasets.load_dataset(dataset_args['dataset_name'], dataset_args['dataset_second_name'],cache_dir=dataset_args['data_cache_dir'])
# num_labels = dataset['train'].features['label'].num_classes
num_labels = 2

Build Training and validating dataset
Using custom data configuration default
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-d2f27f1c83bf5a44/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


In [11]:
text_col_name = 'text'
print(dataset)
print(dataset['train'].features)
print('train size {}'.format(len(dataset['train'])))

print('Train dataset stat:')
text_utils.text_stat([example[text_col_name] for example in dataset['train']])
dataset['train'][1]

print('test size {}'.format(len(dataset['test'])))
print('Test dataset stat:')
text_utils.text_stat([example[text_col_name] for example in dataset['test']])

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'domain', 'task_threshold', 'use'],
        num_rows: 119745
    })
    test: Dataset({
        features: ['text', 'label', 'domain', 'task_threshold', 'use'],
        num_rows: 18627
    })
})
{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'domain': Value(dtype='string', id=None), 'task_threshold': Value(dtype='string', id=None), 'use': Value(dtype='string', id=None)}
train size 119745
Train dataset stat:
Min length: 3, Max length: 19934, Avg length: 526.0338385736356
test size 18627
Test dataset stat:
Min length: 5, Max length: 14475, Avg length: 600.5344392548451


In [12]:
dataset['test'][0]

{'text': "old dvd 1997 looks better than new hd version . i've compared two , first dvd good color saturation , dark blacks looked right heavy red brown palette . hd version blue-green haze covering everything picture too bright . they also removed film grain made image look smooth creamy . sound quality better now , though , image course higher resolution .",
 'label': -1,
 'domain': 'dvd',
 'task_threshold': 't4',
 'use': 'dev'}

In [7]:
def label_select(dataset, per_label_num=32, label_name='label', label_is_index=True):
    lds = []
    for i, name in enumerate(dataset.features[label_name].names):
        label = i if label_is_index else name
        ld = dataset.filter(lambda example: example[label_name]==label).shuffle().select(range(per_label_num))
        lds.append(ld)
    ls = datasets.concatenate_datasets(lds)
    return ls
select_train_dataset = label_select(dataset['train'], dataset_args['per_class_train_size'], label_name='label-coarse')
dataset['train'] = select_train_dataset

Loading cached processed dataset at /home/ubuntu/likun/huggingface_dataset/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-b246349c0734d46b.arrow
Loading cached shuffled indices for dataset at /home/ubuntu/likun/huggingface_dataset/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-97391c367abf3261.arrow
Loading cached processed dataset at /home/ubuntu/likun/huggingface_dataset/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-0378c94e6cde7404.arrow
Loading cached processed dataset at /home/ubuntu/likun/huggingface_dataset/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-033bf1ce24c5d8a3.arrow
Loading cached processed dataset at /home/ubuntu/likun/huggingface_dataset/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-e2958d0a3396289c.arrow
Loading cached processed dataset at /home/ubuntu/

In [8]:
if dataset_args['shuffle']:
    dataset = dataset.shuffle()

### 将文本变换成prompt-based

In [9]:
template = '{}:{}'
mask_token = '<mask>'
label_names = dataset['train'].features['label-coarse'].names
dataset = dataset.map(lambda example: {'labels': template.format(label_names[example['label-coarse']], example['text'])})
dataset = dataset.map(lambda example: {'text': template.format(mask_token, example['text'])})
dataset = dataset.map(lambda example: example, remove_columns=['label-coarse', 'label-fine'])

In [10]:
# dataset = dataset.map(lambda example: {'label': example['label-coarse']}, remove_columns=['label-coarse', 'label-fine'])
dataset['train']['text'][0]

'<mask>:What presidential administration challenged Americans to explore The New Frontier ?'

In [11]:
if dataset_args['val_source'] == 'no':
    train_dataset = dataset['train'].select(range(dataset_args['train_size']))
    test_dataset = dataset['test'].select(range(dataset_args['test_size']))
    val_dataset = None
elif dataset_args['val_source'] == 'train':
    train_dataset = dataset['train'].train_test_split(train_size=dataset_args['train_size'],
                                                      test_size=dataset_args['val_size'])
    train_dataset, val_dataset = train_dataset['train'], train_dataset['test']
    test_dataset = dataset['test'].select(range(dataset_args['test_size']))
elif dataset_args['val_source'] == 'test':
    train_dataset = dataset['train'].select(range(dataset_args['train_size']))
    val_dataset = dataset['test'].select(range(dataset_args['val_size']))
    test_dataset = dataset['test'].select(range(dataset_args['test_size']))
elif dataset_args['val_source'] == 'val':
    train_dataset = dataset['train'].select(range(dataset_args['train_size']))
    val_dataset = dataset['validation'].select(range(dataset_args['val_size']))
    test_dataset = dataset['test'].select(range(dataset_args['test_size']))
else:
    raise Exception("Val source should be 'no', 'train', 'test', 'val' ")

### 文本编码

In [12]:
def head_tail_encode(examples):
    examples['text'] = list(map(lambda t: t[:128] + t[-382:] if len(t) > 510 else t,examples['text']))
    return tokenizer(examples['text'] , max_length=dataset_args['max_length'], truncation=True, padding='max_length')

def standard_encode(examples):
    return tokenizer(examples['text'] , max_length=dataset_args['max_length'], truncation=True, padding='max_length')

def mask_lm_encode(examples):
    res = tokenizer(examples['text'] , max_length=dataset_args['max_length'], truncation=True, padding='max_length')
    res['labels'] = tokenizer(examples['labels'] , max_length=dataset_args['max_length'], truncation=True, padding='max_length')['input_ids']
    return res

In [13]:
encode_func = mask_lm_encode
train_dataset = train_dataset.map(encode_func, batched=True)

if dataset_args['val_source'] != 'no':
    val_dataset = val_dataset.map(encode_func, batched=True)

test_dataset = test_dataset.map(encode_func, batched=True)

In [14]:
logger.critical("Setup the training environment")
model = AutoModelForMaskedLM.from_pretrained(base_pre_trained_model_path)
# model = AutoModelForSequenceClassification.from_pretrained(base_pre_trained_model_path,
#                                                            num_labels=num_labels,
#                                                            output_attentions=False,
#                                                            output_hidden_states=False)
model.config.return_dict = True

Setup the training environment
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at /home/ubuntu/likun/nlp_pretrained/roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def mask_predict_metrics(predict_res):
    mask_id = tokenizer.get_vocab()[mask_token]
    mask_index = [ids.index(mask_id) for ids in test_dataset['input_ids']]
    predict_ids = predict_res.predictions.argmax(2)
    y_predict = predict_ids[range(len(mask_index)), mask_index]
    y_true = predict_res.label_ids[range(len(mask_index)), mask_index]
    return metrics.base_classify_metrics(y_true, y_predict)

def iter_test_eval(test_dataset, chunk_size=32):
    start_index = 0
    end_index = 0
    mask_id = tokenizer.get_vocab()[mask_token]
    y_predict = []
    y_true = []
    while end_index < len(test_dataset):
        start_index = end_index
        end_index = min(end_index + chunk_size, len(test_dataset))
        select_dataset = test_dataset.select(range(start_index, end_index))
        predict_res = trainer.predict(select_dataset)
        mask_index = [ids.index(mask_id) for ids in select_dataset['input_ids']]
        predict_ids = predict_res.predictions.argmax(2)
        y_predict.extend(predict_ids[range(len(mask_index)), mask_index])
        y_true.extend(predict_res.label_ids[range(len(mask_index)), mask_index])
        print("Process end index: {}".format(end_index))
    mres = {'eval_{}'.format(k): v for k, v in metrics.base_classify_metrics(y_true, y_predict).items()}
    return mres

In [16]:
num_train_epochs = 100
train_batch_size = 16
if dataset_args['val_source'] == 'no':
    eval_steps = 1000000
else:
    eval_steps = 30
warmup_steps = int(len(train_dataset) * num_train_epochs // train_batch_size * 0.1)
training_args = TrainingArguments(
    output_dir='/home/ubuntu/likun/nlp_save_kernels/{}'.format(mlflow_tags['mlflow.runName']),  # output directory
    num_train_epochs=num_train_epochs,  # total number of training epochs
    per_device_train_batch_size=train_batch_size,  # batch size per device during training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=warmup_steps,  # number of warmup steps for learning rate scheduler
    weight_decay=0.99,  # strength of weight decay
    logging_dir='/home/ubuntu/likun/nlp_training_logs/{}'.format(mlflow_tags['mlflow.runName']),  # directory for storing logs
    logging_steps=10,
    learning_rate=5e-5,
    seed=44,
    no_cuda=False,
    eval_steps=eval_steps,
    evaluate_during_training=True
)
train_params = {k: v for k, v in training_args.__dict__.items() if (isinstance(v, int) or isinstance(v, float)) and not isinstance(v, bool)}
trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,  # evaluation dataset
    compute_metrics=None,
)

In [17]:
with mlflow.start_run():
    logger.critical("Start to train")
    train_res = trainer.train()
    
    if dataset_args['val_source'] != 'no':
        logger.critical("Start to evaluate")
        eval_res = trainer.evaluate()
    
    logger.critical("Start to test")
#     test_res = trainer.predict(test_dataset).metrics
    test_res = iter_test_eval(test_dataset)
    
    # 记录实验参数
    mlflow.set_tags(mlflow_tags)
    mlflow.log_params(train_params)

    # 记录测试的评估指标
    mlflow.log_metric("train_loss", train_res.training_loss)
    if dataset_args['val_source'] != 'no':
        mlflow.log_metrics(eval_res)
    mlflow.log_metrics({k.replace('eval', 'test'): v for k, v in test_res.items()})
    

Start to train


/home/ubuntu/anaconda3/envs/torch1.3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 22.11819076538086, 'learning_rate': 4.166666666666667e-06, 'epoch': 1.6666666666666665, 'step': 10}



{'loss': 14.96749668121338, 'learning_rate': 8.333333333333334e-06, 'epoch': 3.3333333333333335, 'step': 20}



{'loss': 12.597815895080567, 'learning_rate': 1.25e-05, 'epoch': 5.0, 'step': 30}



...


Process end index: 384



Process end index: 416



Process end index: 448



Process end index: 480



Process end index: 500


In [19]:
# logger.critical("Save the model and config")
# trainer.save_model()
# tokenizer.save_vocabulary(training_args.output_dir)
# logger.critical("Experiment Finnished")
test_res

{'eval_error_rate': 0.16600000000000004,
 'eval_accuracy': 0.834,
 'eval_precision': 0.7850103190604076,
 'eval_recall': 0.8608103391071836,
 'eval_f1_score': 0.7745530999472262}